In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.tabular import *

In [3]:
from fastai import tabular

In [4]:
# import sys
# sys.path.append("../common")

In [5]:
from exp.nb_ import *

In [6]:
# path=Path('../data/other/')

In [7]:
df = pd.read_csv(path/'train.csv.zip', compression='zip')

In [8]:
df.sample(5)

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
579,3,7,1,6,3,2,3,11,2,8,0
6186,3,12,4,9,1,13,1,7,2,1,0
6420,2,6,4,3,2,13,4,10,4,12,0
19868,4,11,4,9,3,6,1,2,4,5,0
10820,2,10,1,11,4,9,2,3,3,6,0


In [9]:
suits = {1:'Hearts', 2:'Spades', 3:'Diamonds', 4:'Clubs'}

In [10]:
cards = {1:'Ace', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9', 10:'10', 11:'Jack', 12:'Queen', 13:'King'}

In [11]:
hands =  {0: 'Nothing', 1:  'Pair', 2: 'Two pairs', 3: 'Three of a kind',
          4: 'Straight', 5: 'Flush', 6: 'Full house', 7: 'Four of a kind',
          8: 'Straight flush', 9: 'Royal flush'}

In [12]:
df = df.replace({'C1':cards, 'C2':cards, 'C3':cards, 'C4':cards, 'C5':cards})

In [13]:
df = df.replace({'S1':suits, 'S2':suits, 'S3':suits, 'S4':suits, 'S5':suits})

In [14]:
df = df.replace({'hand':hands})

In [15]:
df

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,Clubs,9,Spades,Ace,Spades,2,Clubs,7,Spades,8,Nothing
1,Hearts,4,Diamonds,6,Hearts,Queen,Diamonds,Jack,Spades,7,Nothing
2,Hearts,Jack,Clubs,Ace,Diamonds,7,Clubs,Jack,Spades,Ace,Two pairs
3,Spades,9,Spades,4,Diamonds,6,Hearts,9,Clubs,9,Three of a kind
4,Hearts,8,Spades,4,Spades,Jack,Spades,2,Spades,Ace,Nothing
...,...,...,...,...,...,...,...,...,...,...,...
25005,Clubs,9,Clubs,6,Diamonds,6,Clubs,Queen,Clubs,5,Pair
25006,Diamonds,8,Diamonds,5,Clubs,Jack,Spades,2,Hearts,King,Nothing
25007,Hearts,8,Clubs,5,Diamonds,Jack,Diamonds,2,Spades,King,Nothing
25008,Clubs,Queen,Diamonds,5,Spades,Ace,Spades,7,Clubs,6,Nothing


In [16]:
cat_vars_tpl = ('S1', 'C1', 'S2', 'C2', 'S3', 'C3', 'S4', 'C4', 'S5', 'C5')

In [17]:
cont_vars_tpl = ()

In [18]:
cat_vars = list(cat_vars_tpl)
cont_vars = list(cont_vars_tpl)
all_vars = cat_vars + cont_vars

In [19]:
list_diff(df.columns, cat_vars, cont_vars)

['hand']

In [20]:
dep_var = 'hand'

In [21]:
np.random.seed(1001)
ln = len(df)
valid_idx = np.random.choice(ln, int(ln*0.2), replace=False)

In [22]:
len(valid_idx)

5002

In [23]:
procs=[FillMissing, Categorify, Normalize]

In [24]:
def emb_sz_rule_reduced(n_cat:int)->int: return min(10, round(1.6 * n_cat**0.56))

In [25]:
#monkey pacth embenning rule as 600 floats is too much for our case
tabular.data.emb_sz_rule = emb_sz_rule_reduced

In [26]:
BS = 128

### 85%

In [27]:
layers = [1000, 500, 200]
layers_drop = [0.001, 0.005, 0.01]
emb_drop = 0.01
cycles = 60
w_decay = 0.01
max_lr = 1e-3

In [29]:
data = (TabularList.from_df(df, path=path, cat_names=cat_vars, cont_names=cont_vars, procs=procs)
        .split_by_idx(valid_idx)
        .label_from_df(cols=dep_var, label_cls=CategoryList)
        .databunch(bs=BS))

In [30]:
learn = None
np.random.seed(1001)
learn = tabular_learner(data, 
                        layers=layers, 
                        ps=layers_drop, 
                        emb_drop=emb_drop, 
                        metrics=accuracy,
                        callback_fns=[CSVLogger])

In [31]:
max_lr = 3e-3

In [32]:
learn.fit_one_cycle(cyc_len=cycles, max_lr=max_lr, wd=w_decay)

epoch,train_loss,valid_loss,accuracy,time
0,2.205918,2.112571,0.401439,00:01
1,1.838952,1.732437,0.563575,00:01
2,1.309539,1.130222,0.666933,00:01
3,0.813635,0.750517,0.730308,00:01
4,0.678973,0.668179,0.736505,00:01
5,0.605285,0.624136,0.764894,00:01
6,0.583843,0.585979,0.784286,00:01
7,0.578587,0.574682,0.783886,00:01
8,0.555972,0.589546,0.765094,00:01
9,0.555005,0.550606,0.795082,00:01


In [33]:
learn = None
np.random.seed(1001)
learn = tabular_learner(data, 
                        layers=layers, 
                        ps=layers_drop, 
                        emb_drop=emb_drop, 
                        metrics=accuracy,
                        callback_fns=[CSVLogger])

In [34]:
max_lr = 1e-3

In [35]:
learn.fit_one_cycle(cyc_len=cycles, max_lr=max_lr, wd=w_decay)

epoch,train_loss,valid_loss,accuracy,time
0,2.331994,2.298419,0.194322,00:01
1,2.194543,2.170036,0.338065,00:01
2,2.021234,1.974362,0.478009,00:01
3,1.721586,1.634895,0.589564,00:01
4,1.334806,1.243646,0.651939,00:01
5,0.949042,0.932930,0.693722,00:01
6,0.753170,0.758099,0.713315,00:01
7,0.648477,0.701769,0.729308,00:01
8,0.584061,0.622591,0.758097,00:01
9,0.528725,0.572585,0.781887,00:01


In [36]:
learn.save('no_reg_save_0857')

In [37]:
learn.export('no_reg_0857')

### 91%

In [38]:
layers = [1000, 500, 200]
layers_drop = [0, 0, 0]
emb_drop = 0
cycles = 60
w_decay = 0
max_lr = 1e-3

In [39]:
data = (TabularList.from_df(df, path=path, cat_names=cat_vars, cont_names=cont_vars, procs=procs)
        .split_by_idx(valid_idx)
        .label_from_df(cols=dep_var, label_cls=CategoryList)
        .databunch(bs=BS))

In [40]:
np.random.seed(1001)
learn = tabular_learner(data, 
                        layers=layers, 
                        ps=layers_drop, 
                        emb_drop=emb_drop, 
                        metrics=accuracy,
                        callback_fns=[CSVLogger])

In [41]:
learn.fit_one_cycle(cyc_len=cycles, max_lr=max_lr, wd=w_decay)

epoch,train_loss,valid_loss,accuracy,time
0,2.313908,2.273549,0.196921,00:01
1,2.172157,2.179061,0.348061,00:01
2,1.973504,2.007226,0.481807,00:01
3,1.689038,1.660626,0.580768,00:01
4,1.307421,1.237087,0.642143,00:01
5,0.935313,0.906835,0.689324,00:01
6,0.720014,0.736957,0.729108,00:01
7,0.615489,0.668774,0.732907,00:01
8,0.557820,0.615824,0.761295,00:01
9,0.527487,0.616045,0.759296,00:01


In [42]:
learn.save('no_reg_save_0908')
learn.export('no_reg_0908')

In [43]:
max_lr = 1e-4

In [44]:
learn.fit_one_cycle(cyc_len=cycles, max_lr=max_lr, wd=w_decay)

epoch,train_loss,valid_loss,accuracy,time
0,0.002346,0.501966,0.906237,00:01
1,0.001800,0.484959,0.907237,00:01
2,0.002740,0.503099,0.906837,00:01
3,0.002739,0.516548,0.904238,00:01
4,0.002884,0.490764,0.907237,00:01
5,0.006304,0.546647,0.901639,00:01
6,0.008227,0.532048,0.904238,00:01
7,0.011213,0.539905,0.899240,00:01
8,0.014906,0.552567,0.895242,00:01
9,0.022929,0.613138,0.890244,00:01


In [45]:
learn.save('no_reg_save_0917')
learn.export('no_reg_0917')

### Test

In [46]:
test_df = ld_var(name='test_df', path=path)

In [47]:
test_df.sample(5)

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,cards,new_hand
679436,679437,Diamonds,10,Spades,2,Hearts,King,Clubs,Jack,Clubs,3,"[2, 3, 10, 11, 13]",Nothing
607096,607097,Diamonds,3,Spades,Ace,Clubs,8,Hearts,6,Diamonds,8,"[1, 3, 6, 8, 8, 14]",Pair
690751,690752,Spades,Queen,Diamonds,Jack,Hearts,Queen,Spades,8,Hearts,Jack,"[8, 11, 11, 12, 12]",Two pairs
887951,887952,Hearts,8,Hearts,2,Hearts,6,Clubs,7,Hearts,3,"[2, 3, 6, 7, 8]",Nothing
667297,667298,Clubs,Ace,Hearts,9,Hearts,3,Spades,Jack,Spades,Queen,"[1, 3, 9, 11, 12, 14]",Nothing


In [48]:
learn_test = copy(learn)

In [49]:
n = 100000

In [50]:
np.random.seed(1001)
preds = get_cust_preds(df=test_df.iloc[:n], learn=learn_test, bs=BS)

In [51]:
y_hat = np.argmax(preds, axis = 1)

In [52]:
len(y_hat)

100000

In [53]:
c2i = learn_test.data.train_ds.c2i
c2i

{'Flush': 0,
 'Four of a kind': 1,
 'Full house': 2,
 'Nothing': 3,
 'Pair': 4,
 'Royal flush': 5,
 'Straight': 6,
 'Straight flush': 7,
 'Three of a kind': 8,
 'Two pairs': 9}

In [54]:
y = test_df.iloc[:n]['new_hand']
y = y.replace(c2i)

In [55]:
tr = np.count_nonzero(y==y_hat)

In [56]:
accuracy = tr/len(y)

In [57]:
accuracy

0.91744

### 91%